# TODO

- requirements.txt
- git 
- cleanup notebook
- move old notebooks to archive
- cleanup scripts


(based on https://towardsdatascience.com/simple-bert-using-tensorflow-2-0-132cb19e9b22)

# BERT Embeddings with TensorFlow 2.0
With the new release of TensorFlow, this Notebook aims to show a simple use of the BERT model.
- See BERT on paper: https://arxiv.org/pdf/1810.04805.pdf
- See BERT on GitHub: https://github.com/google-research/bert
- See BERT on TensorHub: https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1
- See 'old' use of BERT for comparison: https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb

## Update TF
We need Tensorflow 2.0 and TensorHub 0.7 for this Colab

In [2]:
!pip install tensorflow==2.0
!pip install tensorflow_hub==0.7
!pip install bert-for-tf2
!pip install sentencepiece
!pip install pandas

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

TF version:  2.0.0
Hub version:  0.7.0


If TensorFlow Hub is not 0.7 yet on release, use dev:



In [3]:
# !pip install tf-hub-nightly

You are using pip version 19.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
# hub.__version__

'0.8.0.dev'

In [5]:
bert

<module 'bert' from '/Users/ivp/opt/anaconda3/envs/py36bert/lib/python3.6/site-packages/bert/__init__.py'>

## Import modules

In [4]:
import tensorflow_hub as hub
import tensorflow as tf
import bert
FullTokenizer = bert.bert_tokenization.FullTokenizer
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
import math

Building model using tf.keras and hub. from sentences to embeddings.

Inputs:
 - input token ids (tokenizer converts tokens using vocab file)
 - input masks (1 for useful tokens, 0 for padding)
 - segment ids (for 2 text training: 0 for the first one, 1 for the second one)

Outputs:
 - pooled_output of shape `[batch_size, 768]` with representations for the entire input sequences
 - sequence_output of shape `[batch_size, max_seq_length, 768]` with representations for each input token (in context)

In [6]:
max_seq_length = 128  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

In [7]:
model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

Generating segments and masks based on the original BERT

In [8]:
# See BERT paper: https://arxiv.org/pdf/1810.04805.pdf
# And BERT implementation convert_single_example() at https://github.com/google-research/bert/blob/master/run_classifier.py

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

Import tokenizer using the original vocab file

In [9]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

## Prepare data

In [10]:
import pandas as pd

In [11]:
train = pd.read_csv("./data/input/train.csv")
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [12]:
len(train)

7613

In [13]:
def tokenize(text):
    tokens = tokenizer.tokenize(text)
    tokens = ["[CLS]"] + tokens + ["[SEP]"]
    return tokens


In [15]:
def process_tokens(text):
    #TODO tags to separate column
    #TODO strip hash from tags in text
    return text

In [200]:
def vectorize(text):
    tokens = tokenizer.tokenize(text)
    tokens = ["[CLS]"] + tokens + ["[SEP]"]
    
    input_ids = get_ids(tokens, tokenizer, max_seq_length)
    input_masks = get_masks(tokens, max_seq_length)
    input_segments = get_segments(tokens, max_seq_length)
    
    print('tokens')
    print(tokens)
    
    print('input_ids')
    print(input_ids)
    
    print('input_masks')
    print(input_masks)
    
    print('input_segments')
    print(input_segments)
    
    pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])
    
    print('pool_embs')
    print(pool_embs)
    return pool_embs
    
    

In [196]:
#train["tokenized"] = train["text"].map(tokenize)
train_small = train.sample(20)

In [201]:
train_small["vectorized"] = train_small["text"].map(vectorize)

tokens
['[CLS]', 'i', 'feel', 'like', 'death', '[SEP]']
input_ids
[101, 1045, 2514, 2066, 2331, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
input_masks
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
input_segments
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

pool_embs
[[-0.7512065  -0.3745033  -0.796485    0.67455286  0.5847419  -0.1344934
   0.7825802   0.24072275 -0.73699456 -0.99990875 -0.4312257   0.8284555
   0.97557586  0.6188739   0.8784026  -0.6551231  -0.00942109 -0.5613425
   0.34108564 -0.01411205  0.7477546   0.9999562  -0.00328511  0.40762892
   0.4474513   0.9391002  -0.7103247   0.92100483  0.9196148   0.7179664
  -0.642501    0.24249955 -0.9858255  -0.26978937 -0.8403717  -0.9884022
   0.3917401  -0.5575866  -0.17680386  0.22678035 -0.8568567   0.33631316
   0.9998401   0.18967538  0.5593904  -0.3037029  -0.9999972   0.2570781
  -0.8495009   0.72901666  0.8517841   0.70479804  0.13306242  0.42830518
   0.5672663  -0.25360876 -0.18288147  0.18665066 -0.20588969 -0.45225736
  -0.6351158   0.49019063 -0.8598599  -0.730441    0.81551427  0.71303296
  -0.24634045 -0.28022555 -0.00387213 -0.02118942  0.60719854  0.16781686
  -0.22133265 -0.81251097  0.40269056  0.29362494 -0.69367385  1.
  -0.19469953 -0.96768737  0.8215798   0.6

pool_embs
[[-7.94933975e-01 -3.86959791e-01 -7.13524580e-01  5.45427740e-01
   5.50691724e-01 -7.39176795e-02  6.80218935e-01  2.89937347e-01
  -6.03027046e-01 -9.99911547e-01 -1.58009082e-01  7.70668507e-01
   9.73620594e-01  3.12182844e-01  8.89070988e-01 -5.43737769e-01
  -1.72533974e-01 -5.24981201e-01  2.50746906e-01 -4.46525402e-02
   6.50856316e-01  9.99932706e-01  2.76773363e-01  2.00789943e-01
   5.44991016e-01  9.47788477e-01 -7.56929040e-01  9.23008919e-01
   9.28007901e-01  6.24046147e-01 -5.90048671e-01  1.66957170e-01
  -9.79629338e-01 -1.97988540e-01 -8.21066499e-01 -9.88156199e-01
   3.72515738e-01 -7.02783525e-01  1.44813046e-01  3.24492566e-02
  -8.54885161e-01  1.80978015e-01  9.99862790e-01  1.14166617e-01
   3.81171733e-01 -2.93817699e-01 -9.99998033e-01  2.07311437e-01
  -7.43241966e-01  6.65182054e-01  5.23402512e-01  4.98022258e-01
   9.54833254e-02  3.91865104e-01  4.34700578e-01  7.37305656e-02
  -1.14789918e-01  5.45905270e-02 -1.54966563e-01 -5.30870497e-01


pool_embs
[[-0.79410416 -0.5618757  -0.9185792   0.6742318   0.6416583  -0.26774672
   0.5080239   0.3734727  -0.7734777  -0.9999401  -0.30294663  0.84489286
   0.9749626   0.64525807  0.8530126  -0.29790813  0.108216   -0.58494604
   0.4071072   0.27024567  0.7514531   0.9999791   0.12646078  0.4193221
   0.6482002   0.9682876  -0.70624876  0.84533393  0.9299355   0.7069205
  -0.4635686   0.3859835  -0.9827078  -0.44573694 -0.902648   -0.9849921
   0.44681558 -0.7169302  -0.11293156  0.00320358 -0.80083174  0.473165
   0.99990225  0.25400126  0.3337163  -0.2726601  -0.99999845  0.31161836
  -0.78482515  0.8637586   0.6932623   0.800699    0.27928212  0.43555996
   0.52541685 -0.21407124  0.07370264  0.13127145 -0.29397553 -0.6425146
  -0.6586444   0.51960546 -0.83616817 -0.8839766   0.79229903  0.79453546
  -0.28803188 -0.4428562  -0.17340843  0.00336085  0.6964855   0.3796438
  -0.22470894 -0.886401    0.53098005  0.44010508 -0.6887144   1.
  -0.31535995 -0.96798766  0.75343806  0.74

pool_embs
[[-0.8601618  -0.44267207 -0.733696    0.66939133  0.25950336 -0.17130677
   0.73825353  0.35366595 -0.58040303 -0.99998534 -0.15760395  0.81772685
   0.9882388   0.34847093  0.92893356 -0.3297863   0.22121833 -0.57259774
   0.4650331   0.14429303  0.7467964   0.99979097  0.35402164  0.32563016
   0.62136245  0.94673204 -0.6531518   0.92503476  0.95996356  0.7245461
  -0.49344602  0.3497599  -0.9899797  -0.3427343  -0.76061463 -0.994844
   0.36662206 -0.78245074 -0.03168086 -0.04442457 -0.8315819   0.36916867
   0.99993676  0.3194848   0.21811418 -0.28679562 -0.9999982   0.34712166
  -0.8431781   0.6791967   0.4163692   0.65489143  0.2062325   0.4050987
   0.44294903  0.00416405 -0.02047485  0.11560754 -0.27374902 -0.6069846
  -0.66085607  0.48869225 -0.6762799  -0.9280711   0.6185132   0.54257894
  -0.22425264 -0.37734047 -0.09434104 -0.04748678  0.78912854  0.29786977
  -0.05091195 -0.8741236   0.33969826  0.2938963  -0.6472524   1.
  -0.37531185 -0.9816845   0.5041948   0.

...

pool_embs
[[-0.8353538  -0.6483816  -0.9697386   0.8062788   0.8189162  -0.3465643
   0.6981825   0.5412377  -0.88496065 -0.9999891  -0.6235822   0.94586295
   0.97824365  0.7133583   0.84009373 -0.6943513  -0.03871446 -0.6403623
   0.3966756   0.24685839  0.7771116   0.99999964 -0.18151835  0.5073021
   0.69904226  0.98747605 -0.7353643   0.9017067   0.9537522   0.7327165
  -0.58737844  0.5481033  -0.9866259  -0.47407666 -0.9785195  -0.989635
   0.5975326  -0.67789626 -0.22556673 -0.0598511  -0.7916391   0.55287224
   0.99998796  0.32776064  0.60619044 -0.42236173 -1.          0.4322348
  -0.748763    0.9571915   0.89317816  0.9611009   0.3948556   0.563408
   0.6822025  -0.41251537  0.11607067  0.31523514 -0.41249177 -0.70309484
  -0.7077178   0.6451348  -0.9256476  -0.8651774   0.9410346   0.9223217
  -0.44799808 -0.5410659  -0.22269812  0.00158007  0.7554578   0.44568738
  -0.48573554 -0.88702494  0.7891611   0.46437332 -0.7005062   1.
  -0.6123695  -0.9674366   0.9116977   0.89665

pool_embs
[[-0.6555345  -0.58764124 -0.95771134  0.58586323  0.70932937 -0.16707811
   0.6885501   0.44615456 -0.8252384  -0.9999775  -0.51163304  0.8715034
   0.9651833   0.7460572   0.84225154 -0.42678756  0.31502306 -0.66754866
   0.4490537   0.2515477   0.7338507   0.99999404 -0.15258303  0.3522985
   0.62103623  0.96704715 -0.74734306  0.8646793   0.9031844   0.76295984
  -0.31318128  0.36450014 -0.98000365 -0.4062189  -0.9636147  -0.988069
   0.49054587 -0.6673934  -0.15195623  0.01461669 -0.8281968   0.4327199
   0.999969    0.38990778  0.44863123 -0.39006913 -0.99999964  0.38238645
  -0.74844575  0.9408101   0.84015864  0.8910804   0.19548663  0.5364622
   0.4201622  -0.13809888  0.11492253  0.2297974  -0.3568282  -0.6686426
  -0.6622042   0.5606752  -0.8896734  -0.8664974   0.91418743  0.9108851
  -0.31679973 -0.46168634 -0.21481647 -0.03445125  0.8093887   0.35273764
  -0.5179805  -0.8755697   0.7478806   0.38634363 -0.7387238   1.
  -0.08479898 -0.9590248   0.9124517   0.868

pool_embs
[[-0.7340493  -0.4986751  -0.95424336  0.7007621   0.76529056 -0.11186396
   0.66840094  0.14589949 -0.84645045 -0.9999609  -0.51669145  0.8658073
   0.9075086   0.71455485  0.7267601  -0.74731725 -0.28384155 -0.57001317
   0.42506242 -0.24541292  0.6401553   0.9999974  -0.10820577  0.2219749
   0.40247077  0.9670688  -0.61543     0.75421196  0.934485    0.74533135
  -0.40841514  0.22277449 -0.9609379  -0.18366414 -0.9589022  -0.9710972
   0.29836118 -0.5543582  -0.01913258 -0.10204758 -0.84249616  0.2579621
   0.9999657  -0.00591816  0.43668148 -0.17774923 -0.9999992   0.358134
  -0.7519812   0.92151606  0.8549865   0.8858056   0.2721121   0.40377524
   0.32937706 -0.5272987   0.13367267  0.19474137 -0.13813159 -0.4780204
  -0.5349413   0.23907878 -0.8611444  -0.8091435   0.91705143  0.9002758
  -0.20340392 -0.38250035 -0.06308538 -0.10143133  0.74523383  0.13570888
  -0.19968876 -0.8072662   0.6447087   0.38869256 -0.6695917   1.
  -0.34114715 -0.9201832   0.9096416   0.818

pool_embs
[[-0.7339612  -0.35334438 -0.56821275  0.5058581   0.26103765 -0.09434488
   0.7830579   0.20022403 -0.2634809  -0.99983907  0.1610762   0.6153227
   0.96874964  0.1288226   0.90485114 -0.47526947 -0.05563907 -0.47835645
   0.22400437 -0.28717828  0.641649    0.99941427  0.41471165  0.2762861
   0.36002156  0.78046274 -0.583675    0.9098071   0.9404728   0.5839178
  -0.59511966  0.10292756 -0.9805091  -0.10418949 -0.6545077  -0.97625047
   0.3017912  -0.5577869   0.09739731 -0.00481054 -0.8359057   0.16319144
   0.9993453  -0.48630497  0.12110841 -0.30419183 -0.999981    0.17963277
  -0.77320206  0.27536762  0.42497897  0.24830267  0.07318702  0.29493847
   0.36679044  0.19107951 -0.23360044  0.04027007 -0.09199264 -0.37720892
  -0.47778982  0.3590537  -0.53353095 -0.8639098   0.26214993  0.3702233
  -0.10929687 -0.18822482 -0.0398371  -0.1621848   0.70185465  0.08292334
   0.00730531 -0.8613441   0.13384056  0.1331333  -0.5939275   1.
  -0.4114698  -0.95324016  0.66636044  0

pool_embs
[[-0.7960956  -0.43929693 -0.9212462   0.7008533   0.57457465 -0.23728053
   0.7079909   0.38225022 -0.73638433 -0.99997896 -0.28663266  0.79290205
   0.9560695   0.6422611   0.79033524 -0.5957887  -0.02352088 -0.5568308
   0.41206095 -0.09516928  0.68774235  0.99997514 -0.00300454  0.29310635
   0.49611396  0.944576   -0.66629535  0.8357058   0.919696    0.74729806
  -0.5492923   0.31452036 -0.97433877 -0.34157705 -0.9317944  -0.97828543
   0.42235538 -0.67414767 -0.07442026 -0.07914824 -0.734657    0.38439146
   0.9999466   0.21084815  0.36536404 -0.2845301  -0.99999875  0.23733509
  -0.8078052   0.884234    0.74069345  0.806478    0.19837138  0.44732374
   0.45746338 -0.06435794 -0.01176443  0.10373944 -0.23860954 -0.589582
  -0.572562    0.4282958  -0.82152706 -0.85764116  0.757911    0.81579
  -0.213855   -0.3921016  -0.07239689 -0.094176    0.77232724  0.26806864
  -0.22835605 -0.832918    0.5873964   0.30009392 -0.69932544  1.
  -0.50673515 -0.94786566  0.8257263   0.7

In [202]:
train_small["vectorized2"] = train_small["vectorized"].map(lambda vec: vec[0])

In [203]:
train_small

,id,keyword,location,text,target,tokenized,vectorized,vectorized2
2099,3015,death,NaN,I feel like death,0,"[[CLS], i, feel, like, death, [SEP]]","[[-0.80855566, -0.17480159, 0.6150435, 0.54300...","[-0.80855566, -0.17480159, 0.6150435, 0.543005..."
7194,10306,weapon,"Des Moines, IA",I... I might buy my esoteric weapon for astrol...,0,"[[CLS], i, ., ., ., i, might, buy, my, es, ##o...","[[-0.7512065, -0.3745033, -0.796485, 0.6745528...","[-0.7512065, -0.3745033, -0.796485, 0.67455286..."
6998,10035,twister,"Plano,TX",@briannafrost Twister with Bill Paxton and Hel...,0,"[[CLS], @, brianna, ##fr, ##ost, twist, ##er, ...","[[-0.794934, -0.3869598, -0.7135246, 0.5454277...","[-0.794934, -0.3869598, -0.7135246, 0.54542774..."
6520,9324,survive,Back East in PA,Help me survive the zombie apocalypse at the Z...,0,"[[CLS], help, me, survive, the, zombie, apocal...","[[-0.79410416, -0.5618757, -0.9185792, 0.67423...","[-0.79410416, -0.5618757, -0.9185792, 0.674231..."
2110,3030,death,NaN,https://t.co/oIfN28HpCS @ArianaGrande @ScreamQ...,0,"[[CLS], https, :, /, /, t, ., co, /, o, ##if, ...","[[-0.8601618, -0.44267207, -0.733696, 0.669391...","[-0.8601618, -0.44267207, -0.733696, 0.6693913..."
4668,6635,inundation,Asia European Continent Korea,MEGALPOLIS Area Petting Party Shiver \nFear In...,0,"[[CLS], mega, ##lp, ##olis, area, pet, ##ting,...","[[-0.8535076, -0.45191422, -0.9583118, 0.74424...","[-0.8535076, -0.45191422, -0.9583118, 0.744244..."
2685,3851,detonation,NaN,Ignition Knock (Detonation) Sensor-Senso Stand...,0,"[[CLS], ignition, knock, (, det, ##ona, ##tion...","[[-0.81778115, -0.5937941, -0.9547165, 0.75713...","[-0.81778115, -0.5937941, -0.9547165, 0.757131..."
5785,8254,rioting,"Vista, CA",http://t.co/wspuXOrEWb Cindy Noonan@CindyNoon...,1,"[[CLS], http, :, /, /, t, ., co, /, w, ##sp, #...","[[-0.70002913, -0.4619904, -0.8255487, 0.58795...","[-0.70002913, -0.4619904, -0.8255487, 0.587958..."
2454,3522,derailment,India,Madhya Pradesh Train Derailment: Village Youth...,1,"[[CLS], madhya, pradesh, train, der, ##ail, ##...","[[-0.86973125, -0.5843514, -0.93907064, 0.6929...","[-0.86973125, -0.5843514, -0.93907064, 0.69296..."
4242,6027,hazardous,"Portland, OR",HAZARD - HAZARDOUS CONDITION at SB I5 FWY AT /...,1,"[[CLS], hazard, -, hazardous, condition, at, s...","[[-0.74912125, -0.41679582, -0.92513233, 0.561...","[-0.74912125, -0.41679582, -0.92513233, 0.5611..."


In [204]:
X = train_small["vectorized2"]

In [205]:
y = train_small["target"]

In [48]:
y

1517    1
4121    1
4885    1
5046    0
5869    0
7410    1
2341    1
1605    0
6572    1
4190    0
1360    1
6421    1
4639    0
2665    0
992     0
6574    0
7318    1
241     0
6863    0
7203    1
1034    0
6039    1
2772    1
3241    1
1502    1
2608    0
1055    0
4921    0
7105    1
2153    0
52      0
4281    1
6182    0
2892    0
6512    0
4271    1
4764    0
6866    1
76      0
5610    1
1202    1
5276    1
2712    0
165     1
7089    0
2281    0
4651    0
6913    1
4298    0
400     0
Name: target, dtype: int64

## Train classifier

In [50]:
!pip install sklearn

Processing /Users/ivp/Library/Caches/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074/sklearn-0.0-py2.py3-none-any.whl
     |████████████████████████████████| 11.1MB 1.9MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/28/5c/cf6a2b65a321c4a209efcdf64c2689efae2cb62661f8f6f4bb28547cf1bf/joblib-0.14.1-py2.py3-none-any.whl
     |████████████████████████████████| 28.5MB 971kB/s eta 0:00:01    |█▋                              | 1.5MB 1.1MB/s eta 0:00:24     |██████████▌                     | 9.3MB 1.9MB/s eta 0:00:11     |████████████████████▎           | 18.0MB 500kB/s eta 0:00:21     |███████████████████████▏        | 20.6MB 1.1MB/s eta 0:00:07


In [52]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [53]:
np.random.seed(0)

In [166]:
clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=42) #TODO params

In [158]:
num_features = len(X.values[0])
num_observations = len(X.values)

In [206]:
X1 = np.vstack(X.values)

X

2099    [-0.80855566, -0.17480159, 0.6150435, 0.543005...
7194    [-0.7512065, -0.3745033, -0.796485, 0.67455286...
6998    [-0.794934, -0.3869598, -0.7135246, 0.54542774...
6520    [-0.79410416, -0.5618757, -0.9185792, 0.674231...
2110    [-0.8601618, -0.44267207, -0.733696, 0.6693913...
4668    [-0.8535076, -0.45191422, -0.9583118, 0.744244...
2685    [-0.81778115, -0.5937941, -0.9547165, 0.757131...
5785    [-0.70002913, -0.4619904, -0.8255487, 0.587958...
2454    [-0.86973125, -0.5843514, -0.93907064, 0.69296...
4242    [-0.74912125, -0.41679582, -0.92513233, 0.5611...
1341    [-0.44977453, -0.35364488, -0.69207877, 0.1985...
3781    [-0.81953764, -0.58276725, -0.9689373, 0.70254...
2339    [-0.837714, -0.5467791, -0.86462814, 0.6815728...
6823    [-0.79401135, -0.5790799, -0.8609709, 0.629670...
744     [-0.6465711, -0.23182005, -0.11193719, 0.33047...
5086    [-0.8353538, -0.6483816, -0.9697386, 0.8062788...
5696    [-0.6555345, -0.58764124, -0.95771134, 0.58586...
917     [-0.73

In [207]:
y1 = np.array(y)
y1.shape

(20,)

In [208]:
clf.fit(X1, y1)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [213]:
from sklearn.metrics import classification_report

classification_report(y1, clf.predict(X1))

'              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00        13\n           1       1.00      1.00      1.00         7\n\n    accuracy                           1.00        20\n   macro avg       1.00      1.00      1.00        20\nweighted avg       1.00      1.00      1.00        20\n'

In [174]:
from sklearn.linear_model import LogisticRegression

In [175]:
clf1 = LogisticRegression(random_state=0).fit(X1, y1)

array([-0.5854641 , -0.1109798 ,  0.64318943,  0.3301566 ,  0.19511104,
       -0.10042731,  0.5637863 ,  0.00491682,  0.6151745 , -0.96530753,
        0.10650389, -0.14090131,  0.9499154 , -0.6630013 ,  0.9190232 ,
       -0.25668764,  0.03586264, -0.18043299,  0.16760683, -0.08451284,
        0.7441108 ,  0.95756346,  0.62049806,  0.20532179,  0.22233012,
       -0.04862578, -0.2509603 ,  0.9260144 ,  0.88870704,  0.7638203 ,
       -0.31339264, -0.06534051, -0.9891656 , -0.14508341, -0.05412244,
       -0.9398649 ,  0.05766717, -0.52636147, -0.12261456,  0.13423985,
       -0.89926237,  0.25668642,  0.9770443 , -0.04444619,  0.6866949 ,
       -0.04937407, -0.9701129 ,  0.03326937, -0.9155027 , -0.7498903 ,
       -0.30440867, -0.37041867, -0.16594136,  0.18358374,  0.23124102,
        0.125948  , -0.1326354 ,  0.12387576,  0.00287984, -0.22597599,
       -0.19442499,  0.25121436,  0.06426264, -0.7184028 , -0.6225497 ,
       -0.84394175, -0.15044953, -0.2688857 , -0.07039488, -0.13

In [176]:
clf1.predict(X1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## TMP

In [98]:
import numpy as np
import sklearn as sk
import sklearn.datasets as skd
import sklearn.ensemble as ske


In [99]:
data = skd.load_boston()

In [101]:
reg = ske.RandomForestRegressor()

In [102]:
X = data['data']
y = data['target']

In [104]:
X.shape

(506, 13)

In [107]:
y.shape

(506,)

In [172]:
reg = ske.RandomForestRegressor()

In [173]:
reg.fit(X, y)

ValueError: setting an array element with a sequence.

## Test BERT embedding generator model

In [18]:
s = "This is a nice sentence."

Tokenizing the sentence

In [19]:
stokens = tokenizer.tokenize(s)

In [13]:
stokens

['this', 'is', 'a', 'nice', 'sentence', '.']

Adding separator tokens according to the paper

In [14]:
stokens = ["[CLS]"] + stokens + ["[SEP]"]

Get the model inputs from the tokens

In [15]:
input_ids = get_ids(stokens, tokenizer, max_seq_length)
input_masks = get_masks(stokens, max_seq_length)
input_segments = get_segments(stokens, max_seq_length)

In [16]:
print(stokens)
print(input_ids)
print(input_masks)
print(input_segments)

['[CLS]', 'this', 'is', 'a', 'nice', 'sentence', '.', '[SEP]']
[101, 2023, 2003, 1037, 3835, 6251, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Generate Embeddings using the pretrained model

In [17]:
pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])

In [26]:
pool_embs.shape

(1, 768)

In [35]:
all_embs[0][3]

array([-4.29854274e-01, -1.90815091e-01,  6.27196550e-01, -1.15128897e-01,
        1.98890507e-01, -3.91458184e-01,  3.71359140e-01,  8.00115228e-01,
        2.53230423e-01, -4.50120121e-01,  4.98352677e-01, -1.41464996e+00,
       -1.56425357e-01,  6.15826964e-01, -4.39771712e-01,  7.44559109e-01,
        5.49566261e-02, -1.13790371e-01,  3.84441018e-01, -3.72881740e-01,
       -5.67890070e-02,  4.79586780e-01, -8.50844502e-01,  5.40503860e-01,
        4.94716048e-01, -1.84022471e-01,  4.14134860e-01,  2.30845138e-01,
        6.79596782e-01, -4.54555228e-02,  8.63599956e-01,  2.27411240e-01,
       -1.12967126e-01, -2.99875200e-01, -1.56602129e-01, -9.37395453e-01,
        2.01657027e-01, -7.21723795e-01, -7.49554455e-01, -3.27985287e-01,
       -6.30497754e-01, -1.74673468e-01,  7.62541592e-01,  2.92123854e-03,
       -6.27930760e-01, -2.94100404e-01, -1.98555142e-01, -3.15150768e-01,
        2.55538642e-01, -5.06481469e-01, -4.74405289e-01, -7.16631234e-01,
        6.24211766e-02, -

## Pooled embedding vs [CLS] as sentence-level representation

Previously, the [CLS] token's embedding were used as sentence-level representation (see the original paper). However, here a pooled embedding were introduced. This part is a short comparison of the two embedding using cosine similarity

In [18]:
def square_rooted(x):
    return math.sqrt(sum([a*a for a in x]))


def cosine_similarity(x,y):
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return numerator/float(denominator)

In [19]:
cosine_similarity(pool_embs[0], all_embs[0][0])

0.02757265801760349

In [0]:
TODO generate embeddings for input data
TODO classify with RF